# Data Loading & Exploration

In [1]:
import polars as pl

data_path = "./data/heart.csv"
df = pl.read_csv(data_path)

In [2]:
df

Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
i64,str,str,i64,i64,i64,str,i64,str,f64,str,i64
40,"""M""","""ATA""",140,289,0,"""Normal""",172,"""N""",0.0,"""Up""",0
49,"""F""","""NAP""",160,180,0,"""Normal""",156,"""N""",1.0,"""Flat""",1
37,"""M""","""ATA""",130,283,0,"""ST""",98,"""N""",0.0,"""Up""",0
48,"""F""","""ASY""",138,214,0,"""Normal""",108,"""Y""",1.5,"""Flat""",1
54,"""M""","""NAP""",150,195,0,"""Normal""",122,"""N""",0.0,"""Up""",0
…,…,…,…,…,…,…,…,…,…,…,…
45,"""M""","""TA""",110,264,0,"""Normal""",132,"""N""",1.2,"""Flat""",1
68,"""M""","""ASY""",144,193,1,"""Normal""",141,"""N""",3.4,"""Flat""",1
57,"""M""","""ASY""",130,131,0,"""Normal""",115,"""Y""",1.2,"""Flat""",1


In [3]:
# Features and Target
X, y = df[:, :-1], df[:, -1]

Observations from Data Wrangler:
1. The target variable "HeartDisease" is fairly well balanced (55/45), hence accuracy is a suitable primary metric.
2. The dataset is relatively small (918 samples), hence we can afford to use nestedCV to obtain robust, unbiased performance metrics.
3. There are no missing values in the dataset, except for physiologically impossible cholesterol readings.
4. 172 samples have cholesterol recorded as 0, [which is not possible](https://www.cdc.gov/cholesterol/about/index.html); these will be treated as missing values and imputed.
5. Column transformations are needed to convert categorical variables into numerical representations.
6. From domain knowledge [we know](https://pmc.ncbi.nlm.nih.gov/articles/PMC6932726/) that "ST Slope" (part of an ECG reading) is an ordinal variable where flat and downsloped ST segment correlates with worse cardiac health outcomes than upsloping ST segment.

In [4]:
# 1. Target class balance
print("Class balance:", y.value_counts().with_columns(pl.col("count") / len(y)))
# 2. Data Size
print(f" \n Data Size: {len(df)}")
# 3. Missing Values
print(f"\n Number of missing values: {df.null_count().to_series().sum()}")
# 4. Cholesterol 0 values
print(f"\n Cholesterol 0 values: {df.filter(pl.col('Cholesterol') == 0).height}")


Class balance: shape: (2, 2)
┌──────────────┬──────────┐
│ HeartDisease ┆ count    │
│ ---          ┆ ---      │
│ i64          ┆ f64      │
╞══════════════╪══════════╡
│ 0            ┆ 0.446623 │
│ 1            ┆ 0.553377 │
└──────────────┴──────────┘
 
 Data Size: 918

 Number of missing values: 0

 Cholesterol 0 values: 172


In [5]:
# Separate columns by type for preprocessing
missing_values_columns = ["Cholesterol"]
binary_columns = ["FastingBS"]
numeric_columns = [column.name for column in X.select(pl.col(pl.Int64, pl.Float64))]
numeric_columns = [
    column
    for column in numeric_columns
    if column not in binary_columns and column not in missing_values_columns
]
onehot_columns = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina"]
ordinal_columns = ["ST_Slope"]

# Linear Regression

Linear regression is a natural first choice for exploratory modeling: it’s fast, simple, interpretable, and provides a solid baseline.

## Pipeline Set Up

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


# pipeline for numeric columns that need imputation and scaling - Cholesterol
impute_and_scale = Pipeline(
    [
        ("imputer", SimpleImputer(missing_values=0)),
        ("scaler", StandardScaler()),
    ]
)

# pipeline for ordinal columns - ST_Slope
encode_scale_ordinal = Pipeline(
    [
        (
            "ordinal_encoder",
            OrdinalEncoder(
                categories=[["Down", "Flat", "Up"]],
                dtype=int,
            ),
        ),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        (
            "impute_and_scale",
            impute_and_scale,
            missing_values_columns,
        ),
        (
            "categorical_onehot",
            OneHotEncoder(
                drop="first", sparse_output=False
            ),  # drop="first" to make the model more interpretable
            onehot_columns,
        ),
        ("ordinal_encoder_and_scale", encode_scale_ordinal, ordinal_columns),
        ("scaler", StandardScaler(), numeric_columns),
    ],
    remainder="passthrough",
)
preprocessor.set_output(transform="polars")


,transformers,"[('impute_and_scale', ...), ('categorical_onehot', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,0
,strategy,'mean'
,fill_value,None


In [7]:
from sklearn.linear_model import LogisticRegression
import numpy as np

lr = LogisticRegression(solver="lbfgs", penalty="l2")
param_grid = {
    "preprocess__impute_and_scale__imputer__strategy": ["mean", "median"],
    "clf__C": np.power(10.0, np.arange(-4, 4)),  # regularisation strength
}

pipeline = Pipeline(
    [
        ("preprocess", preprocessor),
        ("clf", lr),
    ]
)

## nCV

Nested cross-validation (nCV) is a method that allows us to tune hyperparameters and estimate model performance in a robust way. 
- The **outer loop** performs *k*-fold CV for performance estimation.
- In each of the *k* folds, an *n*-fold CV is run to optimise *m* model configurations defined in the parameter grid — this is the **inner loop**.
- The best model from the inner loop is returned and scored on the outer fold. This is repeated *k* times.
- The average of the outer fold scores is reported as the model performance. 

nCV reduces bias in model selection and performance estimation by repeatedly sampling from the test error distribution. However, it is computationally expensive (~k × n × m model fits), so it’s typically reserved for smaller datasets.

In [8]:
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedKFold

cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

clf = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, cv=cv_inner, scoring="accuracy"
)  # inner loop tunes hyperparameters
outer_cv = cross_validate(
    estimator=clf,
    X=X,
    y=y,
    scoring=["accuracy", "precision", "recall", "f1"],
    return_estimator=True,
    cv=cv_outer,
)  # outer loop estimate performance

In [9]:
res_dict_lr = {}
for key in outer_cv:
    if "test" in key:
        mean_score = outer_cv[key].mean()
        std_score = outer_cv[key].std()
        res_dict_lr[f"{key}_mean"] = mean_score
        res_dict_lr[f"{key}_std"] = std_score
        print(f"mean {key}: {mean_score:.2f} +/- {std_score:.2f}")

mean test_accuracy: 0.85 +/- 0.04
mean test_precision: 0.85 +/- 0.05
mean test_recall: 0.88 +/- 0.04
mean test_f1: 0.86 +/- 0.03


In [10]:
for i in range(5):
    print(outer_cv["estimator"][i].best_params_)

{'clf__C': np.float64(1.0), 'preprocess__impute_and_scale__imputer__strategy': 'mean'}
{'clf__C': np.float64(1.0), 'preprocess__impute_and_scale__imputer__strategy': 'mean'}
{'clf__C': np.float64(0.1), 'preprocess__impute_and_scale__imputer__strategy': 'mean'}
{'clf__C': np.float64(0.1), 'preprocess__impute_and_scale__imputer__strategy': 'mean'}
{'clf__C': np.float64(1.0), 'preprocess__impute_and_scale__imputer__strategy': 'mean'}


## Results
LR performs well and is stable across the folds as indicated by the small std values. For the final model I would pick **C=0.1** with imputer **strategy=mean** as this configuration was the most frequently selected across folds.

# XGBoost

XGBoost is non-linear and more powerful model than LR. It's the golden standard for tabular data when predictive performance is priority. So it's a natural choice of comparison against LR. Simple, interpretable vs high-performance, non-linear.

## Pipeline Set Up

In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ("imputer", SimpleImputer(missing_values=0), missing_values_columns),
        (
            "categorical_onehot",
            OneHotEncoder(drop="if_binary", sparse_output=False),
            onehot_columns,
        ),
        ("categorical_ordinal", OrdinalEncoder(), ordinal_columns),
    ],
    remainder="passthrough",
)
preprocessor.set_output(transform="polars")


,transformers,"[('imputer', ...), ('categorical_onehot', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,0
,strategy,'mean'
,fill_value,None


In [12]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

param_grid = {
    "preprocess__imputer__strategy": ["mean", "median"],
    "clf__max_depth": [3, 5, 7, 10],
    "clf__n_estimators": [100, 200, 300],
}

pipeline = Pipeline(
    [
        ("preprocess", preprocessor),
        ("clf", xgb),
    ]
)


In [13]:
clf = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, cv=cv_inner, scoring="accuracy"
)
outer_cv = cross_validate(
    estimator=clf,
    X=X,
    y=y,
    scoring=["accuracy", "precision", "recall", "f1"],
    cv=cv_outer,
    return_estimator=True,
)

In [14]:
res_dict_xgb = {}
for key in outer_cv:
    if "test" in key:
        mean_score = outer_cv[key].mean()
        std_score = outer_cv[key].std()
        res_dict_xgb[f"{key}_mean"] = mean_score
        res_dict_xgb[f"{key}_std"] = std_score
        print(f"{key}: {mean_score:.2f} +/- {std_score:.2f}")


test_accuracy: 0.87 +/- 0.02
test_precision: 0.87 +/- 0.03
test_recall: 0.89 +/- 0.03
test_f1: 0.88 +/- 0.02


In [15]:
for i in range(5):
    print(outer_cv["estimator"][i].best_params_)


{'clf__max_depth': 5, 'clf__n_estimators': 100, 'preprocess__imputer__strategy': 'median'}
{'clf__max_depth': 5, 'clf__n_estimators': 100, 'preprocess__imputer__strategy': 'mean'}
{'clf__max_depth': 3, 'clf__n_estimators': 200, 'preprocess__imputer__strategy': 'mean'}
{'clf__max_depth': 3, 'clf__n_estimators': 100, 'preprocess__imputer__strategy': 'mean'}
{'clf__max_depth': 10, 'clf__n_estimators': 300, 'preprocess__imputer__strategy': 'mean'}


## Results
Much like LR, XGBoost performs well and is stable across the folds, as indicated by the small std values. Here, for the final model I would pick **n_estimators=100** with imputer **strategy=mean** as this configuration was the most frequently selected across folds for these parameters. I'd choose **max_depth=5** as this gives the model a bit more power than the other commonly selected value of 3.

# Result comparison

In [16]:
metrics = ["accuracy", "precision", "recall", "f1"]

print(f"{'Metric':<10} | {'LR Mean':<8} {'LR Std':<8} | {'XGB Mean':<8} {'XGB Std':<8}")
print("-" * 50)
for m in metrics:
    lr_mean = res_dict_lr[f"test_{m}_mean"]
    lr_std = res_dict_lr[f"test_{m}_std"]
    xgb_mean = res_dict_xgb[f"test_{m}_mean"]
    xgb_std = res_dict_xgb[f"test_{m}_std"]
    print(
        f"{m:<10} | {lr_mean:<8.2g} {lr_std:<8.1g} | {xgb_mean:<8.2g} {xgb_std:<8.1g}"
    )


Metric     | LR Mean  LR Std   | XGB Mean XGB Std 
--------------------------------------------------
accuracy   | 0.85     0.04     | 0.87     0.02    
precision  | 0.85     0.05     | 0.87     0.03    
recall     | 0.88     0.04     | 0.89     0.03    
f1         | 0.86     0.03     | 0.88     0.02    


XGBoost slightly outperforms LR and is the more stable model, as evidenced by its smaller standard deviations. If raw predictive performance were the priority, XGBoost would be the model of choice. 

However, for this small project I will continue with LR. In a medical setting, especially in clinical practice, interpretability outweighs a few percentage points of performance. The difference here is not significant enough to justify losing interpretability, as the score ranges overlap strongly within the standard deviations; for example, the strongest LR model performs as well as the strongest XGBoost model. XGBoost can be analyzed using tools like SHAP values or feature permutations, but that is beyond the scope of this project.